In [8]:
from pathlib import Path
p = Path("tf_model/model.h5")
if p.exists():
    p.unlink()
print("deleted old:", not p.exists())


deleted old: True


In [2]:
import tensorflow as tf
tf.keras.models.save_model(model, "tf_model/model.h5", save_format="h5")


In [1]:
import json, numpy as np
from pathlib import Path

# 1) Load JSON
p = Path("traffic_features_v1.json")  # ipynb가 data 폴더 안이면 이렇게
with open(p, "r", encoding="utf-8") as f:
    raw = json.load(f)

records = raw["data"]
X = np.array([r["features"] for r in records], dtype=np.float32)
y = np.array([r["label"] for r in records], dtype=np.float32)

print("X", X.shape, "y", y.shape, "pos_rate", y.mean())

# 2) Train/test split (stratify 흉내: 간단히 랜덤 + seed)
rng = np.random.default_rng(42)
idx = np.arange(len(X))
rng.shuffle(idx)
split = int(len(X) * 0.8)
tr, te = idx[:split], idx[split:]

X_train, X_test = X[tr], X[te]
y_train, y_test = y[tr], y[te]

# 3) StandardScaler (numpy로 직접)
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)
std[std == 0] = 1.0

X_train_s = (X_train - mean) / std
X_test_s = (X_test - mean) / std

# 4) TF Logistic model
import tensorflow as tf

tf.random.set_seed(42)

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(14,)),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss="binary_crossentropy",
    metrics=[tf.keras.metrics.AUC(name="auc")]
)

# class weight (불균형 대응)
pos = (y_train == 1).sum()
neg = (y_train == 0).sum()
class_weight = {0: 1.0, 1: float(neg / max(pos, 1))}

hist = model.fit(
    X_train_s, y_train,
    validation_data=(X_test_s, y_test),
    epochs=30,
    batch_size=32,
    class_weight=class_weight,
    verbose=1
)

# 5) Save model (Keras2 HDF5 format ONLY)
out_dir = Path("tf_model")
out_dir.mkdir(exist_ok=True)

model.save(out_dir / "model.h5")   # ✅ TFJS 변환용 정답

# 6) Save scaler params for JS
scaler = {
    "mean": mean.tolist(),
    "std": std.tolist()
}
with open(out_dir / "scaler.json", "w", encoding="utf-8") as f:
    json.dump(scaler, f, ensure_ascii=False, indent=2)

print("saved:", out_dir)


X (506, 14) y (506,) pos_rate 0.13241106
Epoch 1/30
13/13 [==============================] - 1s 14ms/step - loss: 1.4758 - auc: 0.5355 - val_loss: 0.6730 - val_auc: 0.6289
Epoch 2/30
13/13 [==============================] - 0s 3ms/step - loss: 1.0381 - auc: 0.7331 - val_loss: 0.5838 - val_auc: 0.7862
Epoch 3/30
13/13 [==============================] - 0s 2ms/step - loss: 0.8227 - auc: 0.8656 - val_loss: 0.5310 - val_auc: 0.8402
Epoch 4/30
13/13 [==============================] - 0s 3ms/step - loss: 0.7031 - auc: 0.9110 - val_loss: 0.4932 - val_auc: 0.9261
Epoch 5/30
13/13 [==============================] - 0s 3ms/step - loss: 0.6326 - auc: 0.9400 - val_loss: 0.4600 - val_auc: 0.9361
Epoch 6/30
13/13 [==============================] - 0s 3ms/step - loss: 0.5871 - auc: 0.9464 - val_loss: 0.4230 - val_auc: 0.9585
Epoch 7/30
13/13 [==============================] - 0s 3ms/step - loss: 0.5475 - auc: 0.9567 - val_loss: 0.3966 - val_auc: 0.9670
Epoch 8/30
13/13 [==============================

In [3]:
import h5py
with h5py.File("tf_model/model.h5", "r") as f:
    print("keras_version in h5:", f.attrs.get("keras_version"))



keras_version in h5: 2.11.0


In [4]:
import sys, tensorflow as tf
import keras
print("python:", sys.executable)
print("tf:", tf.__version__, "tf.keras:", tf.keras.__version__)
print("keras:", keras.__version__)


python: C:\Users\woain\miniconda3\envs\tfjs_fix\python.exe
tf: 2.11.1 tf.keras: 2.11.0
keras: 2.11.0
